# Flood Damage Calculation Library Tutorial

This notebook provides a comprehensive tutorial on how to use the Flood Damage Calculation Library with JRC (Joint Research Centre) data.

## Table of Contents
1. [Introduction](#introduction)
2. [Setup and Installation](#setup)
3. [Basic Usage](#basic-usage)
4. [Understanding the Data](#understanding-data)
5. [Advanced Features](#advanced-features)
6. [Batch Processing](#batch-processing)
7. [Uncertainty Analysis](#uncertainty-analysis)
8. [Regional Comparisons](#regional-comparisons)
9. [Practical Use Cases](#use-cases)
10. [Best Practices](#best-practices)

---

## 1. Introduction {#introduction}

The Flood Damage Calculation Library is a comprehensive Python tool for estimating economic damages caused by floods. It's based on the global damage functions database from the Joint Research Centre (JRC) of the European Commission.

### Key Features:
- **270 damage functions** from JRC (2017)
- **7 geographic regions**: Europe, North America, Central & South America, Asia, Africa, Oceania, Global
- **248 countries** with specific maximum damage values
- **Uncertainty analysis** with confidence intervals
- **Batch processing** for multiple locations

### What you'll learn:
- How to calculate flood damage for individual locations
- How to process multiple locations efficiently
- How to interpret uncertainty in damage estimates
- How to compare damage across different regions
- Real-world applications and best practices

## 2. Setup and Installation {#setup}

First, let's set up the environment and process the JRC data.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

In [ ]:
# Check if JRC data is processed
jrc_data_dir = Path("./processed_jrc_data")

if not jrc_data_dir.exists():
    print("❌ JRC data not found. Please run the following command in terminal:")
    print("python process_jrc_excel.py")
else:
    print("✅ JRC data directory found")
    
    # List processed files
    parquet_files = list(jrc_data_dir.glob("*.parquet"))
    print(f"📁 Found {len(parquet_files)} processed data files:")
    for file in parquet_files:
        print(f"  - {file.name}")

In [ ]:
# Import the flood damage library
try:
    from flood_damage_library import JRCFloodDamageCalculator, FloodDamageCalculator
    print("✅ Flood damage library imported successfully")
except ImportError as e:
    print(f"❌ Error importing library: {e}")
    print("Make sure you're in the correct directory and the library is properly installed")

## 3. Basic Usage {#basic-usage}

Let's start with a simple example of calculating flood damage for a single location.

In [ ]:
# Initialize the JRC calculator
calculator = JRCFloodDamageCalculator(data_directory="./processed_jrc_data")

print("🔧 JRC Calculator initialized successfully")
print(f"📊 Available regions: {calculator.get_available_regions()}")
print(f"🏢 Available building types: {calculator.get_available_building_types()}")
print(f"🌍 Countries with data: {len(calculator.get_countries_with_data('residential'))}")

In [ ]:
# Example 1: Calculate damage for Berlin, Germany
result = calculator.calculate_jrc_damage(
    latitude=52.5200,      # Berlin coordinates
    longitude=13.4050,
    flood_depth=1.5,       # 1.5 meters flood depth
    country_code='DE',     # Germany
    building_type='residential',
    area_m2=120           # 120 square meters
)

print("🏠 Flood Damage Calculation - Berlin, Germany")
print("=" * 50)
print(f"📍 Location: {result['location']['country_name']} ({result['location']['country_code']})")
print(f"🗺️  Region: {result['location']['region']}")
print(f"🌊 Flood depth: {result['flood_parameters']['depth_m']} m")
print(f"🏢 Building type: {result['property_characteristics']['building_type']}")
print(f"📐 Area: {result['property_characteristics']['area_m2']} m²")
print(f"💶 Max damage per m²: €{result['property_characteristics']['max_damage_per_m2_eur']:.2f}")
print(f"💰 Total property value: €{result['property_characteristics']['total_value_eur']:,.2f}")
print(f"📊 Damage ratio: {result['damage_assessment']['damage_ratio']:.1%}")
print(f"💸 Economic damage: €{result['damage_assessment']['economic_damage_eur']:,.2f}")

# Show confidence interval
ci_95 = result['uncertainty_analysis']['confidence_interval_95']
print(f"📈 95% Confidence Interval: €{ci_95['lower_eur']:,.0f} - €{ci_95['upper_eur']:,.0f}")

### 3.1 Coordinates Are Optional! 🌍

**Good news!** You don't always need coordinates. If you know the country code, you can skip coordinates entirely:

In [ ]:
# Method 1: With coordinates (traditional approach)
print("🗺️  Method 1: Using coordinates")
result1 = calculator.calculate_jrc_damage(
    latitude=52.5200,
    longitude=13.4050,
    flood_depth=1.5,
    building_type='residential',
    area_m2=120
)
print(f"   Country inferred: {result1['location']['country_code']}")
print(f"   Damage: €{result1['damage_assessment']['economic_damage_eur']:,.2f}")

print("\n🏴 Method 2: Using country code only (no coordinates!)")
result2 = calculator.calculate_jrc_damage(
    flood_depth=1.5,
    country_code='DE',  # Germany
    building_type='residential',
    area_m2=120
)
print(f"   Country used: {result2['location']['country_code']}")
print(f"   Damage: €{result2['damage_assessment']['economic_damage_eur']:,.2f}")
print(f"   Has coordinates in result: {'latitude' in result2['location']}")

print("\n🚀 Method 3: Simplified API (recommended for country-based calculations)")
result3 = calculator.calculate_damage_by_country(
    flood_depth=1.5,
    country_code='DE',
    building_type='residential',
    area_m2=120
)
print(f"   Country used: {result3['location']['country_code']}")
print(f"   Damage: €{result3['damage_assessment']['economic_damage_eur']:,.2f}")

print("\n✅ All three methods give the same result!")
print(f"   Same damage values: {result1['damage_assessment']['economic_damage_eur'] == result2['damage_assessment']['economic_damage_eur'] == result3['damage_assessment']['economic_damage_eur']}")

### When to use each method:

- **Method 1 (with coordinates)**: When you have GPS data but don't know the country
- **Method 2 (country code)**: When you know the country and want full control
- **Method 3 (simplified API)**: When you know the country and want the cleanest code

**Performance tip**: Methods 2 and 3 are slightly faster since they skip coordinate validation.

### 3.2 All Building Types Supported! 🏗️

The library supports **6 different building types** from the JRC database:

In [ ]:
# Test all building types with the same flood scenario
print("🏗️ Testing all building types - 1.5m flood in Germany (100m²)")
print("=" * 70)

building_types = calculator.get_available_building_types()
results = []

for building_type in building_types:
    result = calculator.calculate_damage_by_country(
        flood_depth=1.5,
        country_code='DE',
        building_type=building_type,
        area_m2=100
    )
    
    damage = result['damage_assessment']['economic_damage_eur']
    max_damage_m2 = result['property_characteristics']['max_damage_per_m2_eur']
    damage_ratio = result['damage_assessment']['damage_ratio']
    
    results.append({
        'Building Type': building_type.title(),
        'Max Damage (€/m²)': f"€{max_damage_m2:.0f}",
        'Damage Ratio': f"{damage_ratio:.1%}",
        'Total Damage': f"€{damage:,.0f}"
    })
    
    print(f"🏢 {building_type.title():13}: €{damage:8,.0f} ({damage_ratio:.1%} of €{max_damage_m2:.0f}/m²)")

print("\n💡 Key insights:")
print("   • Commercial buildings have highest damage per m²")
print("   • Agriculture has lower values (land-based)")
print("   • Transport infrastructure is expensive to replace")
print("   • All types use the same damage curves but different max values")

### Real-world examples by building type:

- **Residential**: Houses, apartments, condominiums
- **Commercial**: Offices, shops, restaurants, hotels
- **Industrial**: Factories, warehouses, manufacturing plants
- **Agriculture**: Farmland, crops, agricultural facilities
- **Infrastructure**: Roads, bridges, utilities, public facilities
- **Transport**: Railways, airports, ports, transport hubs

In [ ]:
# Compare agricultural damage across different countries
print("🌾 Agricultural flood damage comparison (2m flood, 1000m² farmland)")
print("=" * 65)

countries = ['US', 'DE', 'FR', 'BR', 'IN', 'AU', 'JP']
ag_results = []

for country in countries:
    try:
        result = calculator.calculate_damage_by_country(
            flood_depth=2.0,
            country_code=country,
            building_type='agriculture',
            area_m2=1000  # 1000 m² = 0.1 hectare
        )
        
        damage = result['damage_assessment']['economic_damage_eur']
        country_name = result['location']['country_name']
        region = result['location']['region']
        
        print(f"🌍 {country} ({country_name[:15]:15}): €{damage:8,.0f} [{region}]")
        
    except Exception as e:
        print(f"❌ {country}: {e}")

print("\n🚛 Transport infrastructure comparison (1.5m flood, 500m²)")
print("=" * 60)

for country in ['US', 'DE', 'JP', 'GB']:
    try:
        result = calculator.calculate_damage_by_country(
            flood_depth=1.5,
            country_code=country,
            building_type='transport',
            area_m2=500  # 500 m² transport facility
        )
        
        damage = result['damage_assessment']['economic_damage_eur']
        country_name = result['location']['country_name']
        
        print(f"🚄 {country} ({country_name[:15]:15}): €{damage:8,.0f}")
        
    except Exception as e:
        print(f"❌ {country}: {e}")

## 4. Understanding the Data {#understanding-data}

Let's explore the JRC data structure and understand what information is available.

In [ ]:
# Load and examine the damage functions data
damage_functions = pd.read_parquet('./processed_jrc_data/damage_functions_jrc.parquet')

print("📊 JRC Damage Functions Dataset")
print("=" * 40)
print(f"Total records: {len(damage_functions)}")
print(f"Columns: {list(damage_functions.columns)}")
print(f"\nBuilding types: {damage_functions['building_type'].unique()}")
print(f"\nRegions: {damage_functions['region'].unique()}")
print(f"\nDepth range: {damage_functions['depth_m'].min():.1f} - {damage_functions['depth_m'].max():.1f} meters")

# Show sample data
print("\n📋 Sample data (Residential buildings in Europe):")
sample = damage_functions[
    (damage_functions['building_type'] == 'residential_buildings') & 
    (damage_functions['region'] == 'EUROPE')
][['depth_m', 'damage_ratio', 'standard_deviation']]
print(sample.to_string(index=False))

In [ ]:
# Visualize damage functions by region
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('JRC Damage Functions by Region (Residential Buildings)', fontsize=16, fontweight='bold')

regions_to_plot = ['EUROPE', 'North AMERICA', 'ASIA', 'GLOBAL']
colors = ['blue', 'red', 'green', 'orange']

for i, (region, color) in enumerate(zip(regions_to_plot, colors)):
    ax = axes[i//2, i%2]
    
    # Filter data for this region and residential buildings
    region_data = damage_functions[
        (damage_functions['region'] == region) & 
        (damage_functions['building_type'] == 'residential_buildings')
    ]
    
    if not region_data.empty:
        ax.plot(region_data['depth_m'], region_data['damage_ratio'], 
               marker='o', linewidth=2, markersize=6, color=color, label=region)
        ax.set_xlabel('Flood Depth (m)')
        ax.set_ylabel('Damage Ratio')
        ax.set_title(f'{region}')
        ax.grid(True, alpha=0.3)
        ax.set_ylim(0, 1.1)
    else:
        ax.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'{region} (No data)')

plt.tight_layout()
plt.show()

In [ ]:
# Examine maximum damage values by country
max_damage_residential = pd.read_parquet('./processed_jrc_data/max_damage_residential_jrc.parquet')

print("💰 Maximum Damage Values (Residential)")
print("=" * 40)
print(f"Countries with data: {len(max_damage_residential)}")
print(f"Columns: {list(max_damage_residential.columns)}")

# Show top 10 countries by total building damage
top_countries = max_damage_residential.nlargest(10, 'total_building_eur_m2')
print("\n🏆 Top 10 countries by maximum damage per m²:")
for _, row in top_countries.iterrows():
    print(f"  {row['country']:20}: €{row['total_building_eur_m2']:8.2f}/m²")

# Show bottom 10 countries
bottom_countries = max_damage_residential.nsmallest(10, 'total_building_eur_m2')
print("\n📉 Bottom 10 countries by maximum damage per m²:")
for _, row in bottom_countries.iterrows():
    print(f"  {row['country']:20}: €{row['total_building_eur_m2']:8.2f}/m²")

## 5. Advanced Features {#advanced-features}

Let's explore more advanced features of the library, including different building types and automatic region inference.

In [ ]:
# Compare different building types for the same location
location = {'latitude': 52.5200, 'longitude': 13.4050, 'country_code': 'DE'}
building_types = ['residential', 'commercial', 'industrial']
flood_depth = 1.5
area = 100

print("🏢 Building Type Comparison - Berlin, Germany")
print("=" * 60)
print(f"📍 Location: Berlin, Germany")
print(f"🌊 Flood depth: {flood_depth} m")
print(f"📐 Area: {area} m²")
print()

building_results = []

for building_type in building_types:
    result = calculator.calculate_jrc_damage(
        latitude=location['latitude'],
        longitude=location['longitude'],
        flood_depth=flood_depth,
        country_code=location['country_code'],
        building_type=building_type,
        area_m2=area
    )
    
    damage = result['damage_assessment']['economic_damage_eur']
    ratio = result['damage_assessment']['damage_ratio']
    max_damage = result['property_characteristics']['max_damage_per_m2_eur']
    
    building_results.append({
        'type': building_type,
        'damage': damage,
        'ratio': ratio,
        'max_damage_per_m2': max_damage
    })
    
    print(f"{building_type.capitalize():12}: €{damage:8,.0f} ({ratio:5.1%}) - Max: €{max_damage:6.0f}/m²")

# Create comparison chart
df_comparison = pd.DataFrame(building_results)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Damage comparison
ax1.bar(df_comparison['type'], df_comparison['damage'], color=['skyblue', 'lightcoral', 'lightgreen'])
ax1.set_title('Economic Damage by Building Type')
ax1.set_ylabel('Damage (EUR)')
ax1.tick_params(axis='x', rotation=45)

# Damage ratio comparison
ax2.bar(df_comparison['type'], df_comparison['ratio'], color=['skyblue', 'lightcoral', 'lightgreen'])
ax2.set_title('Damage Ratio by Building Type')
ax2.set_ylabel('Damage Ratio')
ax2.tick_params(axis='x', rotation=45)
ax2.set_ylim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
# Test automatic region inference
test_locations = [
    {'name': 'Berlin, Germany', 'lat': 52.52, 'lon': 13.40, 'expected': 'EUROPE'},
    {'name': 'New York, USA', 'lat': 40.71, 'lon': -74.01, 'expected': 'North AMERICA'},
    {'name': 'Tokyo, Japan', 'lat': 35.68, 'lon': 139.65, 'expected': 'ASIA'},
    {'name': 'São Paulo, Brazil', 'lat': -23.55, 'lon': -46.63, 'expected': 'Centr&South_AMERICA'},
    {'name': 'Lagos, Nigeria', 'lat': 6.52, 'lon': 3.38, 'expected': 'AFRICA'},
    {'name': 'Sydney, Australia', 'lat': -33.87, 'lon': 151.21, 'expected': 'OCEANIA'}
]

print("🌍 Automatic Region Inference Test")
print("=" * 50)
print(f"{'Location':20} | {'Inferred Region':20} | {'Expected':20} | {'Match':5}")
print("-" * 70)

for loc in test_locations:
    result = calculator.calculate_jrc_damage(
        latitude=loc['lat'],
        longitude=loc['lon'],
        flood_depth=1.0,
        building_type='residential',
        area_m2=100
    )
    
    inferred_region = result['location']['region']
    match = "✅" if inferred_region == loc['expected'] else "❌"
    
    print(f"{loc['name']:20} | {inferred_region:20} | {loc['expected']:20} | {match:5}")

## 6. Batch Processing {#batch-processing}

For analyzing multiple locations efficiently, the library provides batch processing capabilities.

In [ ]:
# Create a portfolio of properties across Europe
european_portfolio = [
    {
        'name': 'Berlin Office',
        'latitude': 52.5200, 'longitude': 13.4050,
        'country_code': 'DE', 'building_type': 'commercial',
        'area_m2': 500, 'flood_depth': 1.5
    },
    {
        'name': 'Paris Residence',
        'latitude': 48.8566, 'longitude': 2.3522,
        'country_code': 'FR', 'building_type': 'residential',
        'area_m2': 120, 'flood_depth': 2.0
    },
    {
        'name': 'Milan Factory',
        'latitude': 45.4642, 'longitude': 9.1900,
        'country_code': 'IT', 'building_type': 'industrial',
        'area_m2': 1000, 'flood_depth': 1.0
    },
    {
        'name': 'Madrid Center',
        'latitude': 40.4168, 'longitude': -3.7038,
        'country_code': 'ES', 'building_type': 'commercial',
        'area_m2': 300, 'flood_depth': 1.8
    },
    {
        'name': 'Amsterdam Warehouse',
        'latitude': 52.3676, 'longitude': 4.9041,
        'country_code': 'NL', 'building_type': 'industrial',
        'area_m2': 800, 'flood_depth': 2.5
    }
]

# Process the portfolio using batch calculation
batch_results = calculator.calculate_damage_batch_jrc(european_portfolio)

print("🏢 European Property Portfolio Analysis")
print("=" * 70)
print(f"{'Property':20} | {'Type':11} | {'Area':6} | {'Depth':5} | {'Value':12} | {'Damage':12} | {'Ratio':5}")
print("-" * 70)

total_value = 0
total_damage = 0
portfolio_data = []

for i, result in enumerate(batch_results):
    if 'error' not in result:
        prop = european_portfolio[i]
        value = result['property_characteristics']['total_value_eur']
        damage = result['damage_assessment']['economic_damage_eur']
        ratio = result['damage_assessment']['damage_ratio']
        
        total_value += value
        total_damage += damage
        
        portfolio_data.append({
            'name': prop['name'],
            'type': prop['building_type'],
            'area': prop['area_m2'],
            'depth': prop['flood_depth'],
            'value': value,
            'damage': damage,
            'ratio': ratio
        })
        
        print(f"{prop['name']:20} | {prop['building_type']:11} | {prop['area_m2']:4.0f}m² | {prop['flood_depth']:4.1f}m | €{value:9,.0f} | €{damage:9,.0f} | {ratio:5.1%}")
    else:
        print(f"{european_portfolio[i]['name']:20} | ERROR: {result['error']}")

print("-" * 70)
print(f"{'TOTAL':20} | {'':11} | {'':6} | {'':5} | €{total_value:9,.0f} | €{total_damage:9,.0f} | {total_damage/total_value:5.1%}")

print(f"\n📊 Portfolio Summary:")
print(f"  Total portfolio value: €{total_value:,.0f}")
print(f"  Total estimated damage: €{total_damage:,.0f}")
print(f"  Expected loss ratio: {total_damage/total_value:.2%}")

In [ ]:
# Visualize portfolio analysis
df_portfolio = pd.DataFrame(portfolio_data)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('European Property Portfolio Analysis', fontsize=16, fontweight='bold')

# 1. Damage by property
ax1.bar(df_portfolio['name'], df_portfolio['damage'], color='lightcoral')
ax1.set_title('Economic Damage by Property')
ax1.set_ylabel('Damage (EUR)')
ax1.tick_params(axis='x', rotation=45)

# 2. Value vs Damage
ax2.scatter(df_portfolio['value'], df_portfolio['damage'], s=100, alpha=0.7, color='steelblue')
for i, name in enumerate(df_portfolio['name']):
    ax2.annotate(name.split()[0], (df_portfolio['value'].iloc[i], df_portfolio['damage'].iloc[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)
ax2.set_xlabel('Property Value (EUR)')
ax2.set_ylabel('Damage (EUR)')
ax2.set_title('Property Value vs Damage')

# 3. Damage ratio by building type
type_summary = df_portfolio.groupby('type').agg({
    'damage': 'sum',
    'value': 'sum'
}).reset_index()
type_summary['ratio'] = type_summary['damage'] / type_summary['value']

ax3.bar(type_summary['type'], type_summary['ratio'], color=['skyblue', 'lightgreen', 'orange'])
ax3.set_title('Average Damage Ratio by Building Type')
ax3.set_ylabel('Damage Ratio')
ax3.tick_params(axis='x', rotation=45)

# 4. Portfolio composition
ax4.pie(type_summary['value'], labels=type_summary['type'], autopct='%1.1f%%', startangle=90)
ax4.set_title('Portfolio Value Composition')

plt.tight_layout()
plt.show()

## 6.1. Batch Processing by Country (No Coordinates Required)

The library now supports batch processing using only country codes, making it perfect for portfolio analysis when you don't have precise coordinates.

In [ ]:
# Create a global portfolio using only country codes
global_portfolio_scenarios = [
    {
        'name': 'Germany Residential',
        'flood_depth': 1.5,
        'country_code': 'DE',
        'building_type': 'residential',
        'area_m2': 120
    },
    {
        'name': 'France Commercial',
        'flood_depth': 2.0,
        'country_code': 'FR',
        'building_type': 'commercial',
        'area_m2': 500
    },
    {
        'name': 'Italy Industrial',
        'flood_depth': 1.0,
        'country_code': 'IT',
        'building_type': 'industrial',
        'area_m2': 1000
    },
    {
        'name': 'Spain Agriculture',
        'flood_depth': 2.5,
        'country_code': 'ES',
        'building_type': 'agriculture',
        'area_m2': 2000
    },
    {
        'name': 'Netherlands Infrastructure',
        'flood_depth': 1.8,
        'country_code': 'NL',
        'building_type': 'infrastructure',
        'area_m2': 300
    },
    {
        'name': 'UK Transport',
        'flood_depth': 1.2,
        'country_code': 'GB',
        'building_type': 'transport',
        'area_m2': 400
    }
]

print("🌍 Global Portfolio Analysis - Batch by Country")
print("=" * 80)
print("Processing scenarios using only country codes (no coordinates needed)...\n")

# Use the new batch by country function
country_batch_results = calculator.calculate_damage_batch_by_country(global_portfolio_scenarios)

print(f"{'Scenario':25} | {'Country':7} | {'Type':13} | {'Depth':5} | {'Area':6} | {'Damage':12} | {'Status':8}")
print("-" * 80)

total_portfolio_damage = 0
successful_scenarios = 0

for result in country_batch_results:
    scenario = result['input_scenario']
    portfolio_item = global_portfolio_scenarios[result['batch_index']]
    
    if 'error' not in result:
        damage = result['damage_assessment']['economic_damage_eur']
        total_portfolio_damage += damage
        successful_scenarios += 1
        status = "✅ OK"
        
        print(f"{portfolio_item['name']:25} | {scenario['country_code']:7} | {scenario['building_type']:13} | {scenario['flood_depth']:4.1f}m | {scenario['area_m2']:4.0f}m² | €{damage:9,.0f} | {status:8}")
    else:
        status = "❌ ERROR"
        print(f"{portfolio_item['name']:25} | {scenario['country_code']:7} | {scenario['building_type']:13} | {scenario['flood_depth']:4.1f}m | {scenario['area_m2']:4.0f}m² | {'N/A':>9} | {status:8}")
        print(f"   Error: {result['error']}")

print("-" * 80)
print(f"📊 PORTFOLIO SUMMARY:")
print(f"   • Successful calculations: {successful_scenarios}/{len(global_portfolio_scenarios)}")
print(f"   • Total portfolio damage: €{total_portfolio_damage:,.0f}")
print(f"   • Average damage per asset: €{total_portfolio_damage/successful_scenarios:,.0f}")
print(f"\n💡 Advantage: No coordinates needed - perfect for country-level risk assessment!")

### Comparison: Batch with Coordinates vs. Batch by Country

Let's compare both batch processing methods to understand when to use each one.

In [ ]:
import time

# Test scenarios for comparison
test_scenarios = [
    {'flood_depth': 1.5, 'country_code': 'DE', 'building_type': 'residential', 'area_m2': 100},
    {'flood_depth': 2.0, 'country_code': 'FR', 'building_type': 'commercial', 'area_m2': 200},
    {'flood_depth': 1.0, 'country_code': 'IT', 'building_type': 'industrial', 'area_m2': 500}
]

# Method 1: Batch by country (no coordinates)
start_time = time.time()
results_by_country = calculator.calculate_damage_batch_by_country(test_scenarios)
time_by_country = time.time() - start_time

# Method 2: Traditional batch with coordinates
test_scenarios_with_coords = [
    {'latitude': 52.52, 'longitude': 13.40, 'flood_depth': 1.5, 'country_code': 'DE', 'building_type': 'residential', 'area_m2': 100},
    {'latitude': 48.86, 'longitude': 2.35, 'flood_depth': 2.0, 'country_code': 'FR', 'building_type': 'commercial', 'area_m2': 200},
    {'latitude': 41.90, 'longitude': 12.50, 'flood_depth': 1.0, 'country_code': 'IT', 'building_type': 'industrial', 'area_m2': 500}
]

start_time = time.time()
results_with_coords = calculator.calculate_damage_batch_jrc(test_scenarios_with_coords)
time_with_coords = time.time() - start_time

print("⚡ PERFORMANCE COMPARISON")
print("=" * 50)
print(f"Batch by Country:     {time_by_country:.4f} seconds")
print(f"Batch with Coords:    {time_with_coords:.4f} seconds")
print(f"Speed difference:     {abs(time_with_coords - time_by_country):.4f} seconds")

print("\n🎯 WHEN TO USE EACH METHOD:")
print("\n📍 Use Batch by Country when:")
print("   • You only have country-level data")
print("   • Doing high-level risk assessment")
print("   • Analyzing portfolios across countries")
print("   • You want consistent country-level estimates")

print("\n🗺️  Use Batch with Coordinates when:")
print("   • You have precise location data")
print("   • Need location-specific estimates")
print("   • Working with detailed property data")
print("   • Coordinates provide regional specificity")

# Compare results
print("\n💰 DAMAGE COMPARISON:")
print(f"{'Method':20} | {'DE Residential':15} | {'FR Commercial':15} | {'IT Industrial':15}")
print("-" * 70)

damages_by_country = [r['damage_assessment']['economic_damage_eur'] for r in results_by_country if 'error' not in r]
damages_with_coords = [r['damage_assessment']['economic_damage_eur'] for r in results_with_coords if 'error' not in r]

print(f"{'By Country':20} | €{damages_by_country[0]:12,.0f} | €{damages_by_country[1]:12,.0f} | €{damages_by_country[2]:12,.0f}")
print(f"{'With Coordinates':20} | €{damages_with_coords[0]:12,.0f} | €{damages_with_coords[1]:12,.0f} | €{damages_with_coords[2]:12,.0f}")

# Calculate differences
differences = [(abs(d1 - d2) / d1 * 100) for d1, d2 in zip(damages_by_country, damages_with_coords)]
print(f"{'Difference (%)':20} | {differences[0]:12.1f}% | {differences[1]:12.1f}% | {differences[2]:12.1f}%")

## 7. Uncertainty Analysis {#uncertainty-analysis}

Understanding uncertainty in damage estimates is crucial for risk assessment. Let's explore the uncertainty analysis features.

In [ ]:
# Detailed uncertainty analysis for a specific case
uncertainty_case = calculator.calculate_jrc_damage(
    latitude=52.5200,
    longitude=13.4050,
    flood_depth=1.5,
    country_code='DE',
    building_type='residential',
    area_m2=200
)

damage = uncertainty_case['damage_assessment']['economic_damage_eur']
uncertainty = uncertainty_case['uncertainty_analysis']

print("📊 Detailed Uncertainty Analysis")
print("=" * 50)
print(f"📍 Location: Berlin, Germany")
print(f"🏠 Building: Residential, 200m², 1.5m flood depth")
print()
print(f"💰 Estimated damage: €{damage:,.2f}")
print(f"📈 Standard deviation: {uncertainty['standard_deviation_ratio']:.1%}")
print(f"💸 Damage std deviation: €{uncertainty['damage_standard_deviation_eur']:,.2f}")
print()

# Confidence intervals
ci_68 = uncertainty['confidence_interval_68']
ci_95 = uncertainty['confidence_interval_95']

print("🎯 Confidence Intervals:")
print(f"  68% (±1σ): €{ci_68['lower_eur']:,.0f} - €{ci_68['upper_eur']:,.0f}")
print(f"  95% (±2σ): €{ci_95['lower_eur']:,.0f} - €{ci_95['upper_eur']:,.0f}")
print()

# Interpretation
range_68 = ci_68['upper_eur'] - ci_68['lower_eur']
range_95 = ci_95['upper_eur'] - ci_95['lower_eur']

print("📋 Interpretation:")
print(f"  There's a 68% probability the damage is between €{ci_68['lower_eur']:,.0f} and €{ci_68['upper_eur']:,.0f}")
print(f"  There's a 95% probability the damage is between €{ci_95['lower_eur']:,.0f} and €{ci_95['upper_eur']:,.0f}")
print(f"  Uncertainty range (95%): ±€{range_95/2:,.0f} ({range_95/damage:.1%} of estimated damage)")

In [ ]:
# Visualize uncertainty
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. Confidence intervals visualization
scenarios = ['Optimistic\n(P5)', 'Expected', 'Pessimistic\n(P95)']
values = [ci_95['lower_eur'], damage, ci_95['upper_eur']]
colors = ['green', 'blue', 'red']

bars = ax1.bar(scenarios, values, color=colors, alpha=0.7)
ax1.set_title('Damage Scenarios with Uncertainty')
ax1.set_ylabel('Damage (EUR)')

# Add value labels on bars
for bar, value in zip(bars, values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 1000,
             f'€{value:,.0f}', ha='center', va='bottom')

# 2. Normal distribution approximation
std_dev = uncertainty['damage_standard_deviation_eur']
x = np.linspace(damage - 3*std_dev, damage + 3*std_dev, 1000)
y = (1/(std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - damage) / std_dev) ** 2)

ax2.plot(x, y, 'b-', linewidth=2, label='Damage Distribution')
ax2.axvline(damage, color='blue', linestyle='--', label=f'Expected: €{damage:,.0f}')
ax2.axvline(ci_68['lower_eur'], color='orange', linestyle=':', alpha=0.7, label='68% CI')
ax2.axvline(ci_68['upper_eur'], color='orange', linestyle=':', alpha=0.7)
ax2.axvline(ci_95['lower_eur'], color='red', linestyle=':', alpha=0.7, label='95% CI')
ax2.axvline(ci_95['upper_eur'], color='red', linestyle=':', alpha=0.7)

# Fill confidence intervals
x_68 = x[(x >= ci_68['lower_eur']) & (x <= ci_68['upper_eur'])]
y_68 = (1/(std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x_68 - damage) / std_dev) ** 2)
ax2.fill_between(x_68, y_68, alpha=0.3, color='orange', label='68% Confidence')

x_95 = x[(x >= ci_95['lower_eur']) & (x <= ci_95['upper_eur'])]
y_95 = (1/(std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x_95 - damage) / std_dev) ** 2)
ax2.fill_between(x_95, y_95, alpha=0.2, color='red', label='95% Confidence')

ax2.set_xlabel('Damage (EUR)')
ax2.set_ylabel('Probability Density')
ax2.set_title('Damage Uncertainty Distribution')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Regional Comparisons {#regional-comparisons}

Let's compare how the same flood conditions affect different regions around the world.

In [ ]:
# Compare the same asset across different regions
global_comparison = [
    {'name': 'Berlin', 'country': 'Germany', 'lat': 52.52, 'lon': 13.40, 'code': 'DE'},
    {'name': 'New York', 'country': 'USA', 'lat': 40.71, 'lon': -74.01, 'code': 'US'},
    {'name': 'Tokyo', 'country': 'Japan', 'lat': 35.68, 'lon': 139.65, 'code': 'JP'},
    {'name': 'São Paulo', 'country': 'Brazil', 'lat': -23.55, 'lon': -46.63, 'code': 'BR'},
    {'name': 'Lagos', 'country': 'Nigeria', 'lat': 6.52, 'lon': 3.38, 'code': 'NG'},
    {'name': 'Sydney', 'country': 'Australia', 'lat': -33.87, 'lon': 151.21, 'code': 'AU'}
]

# Standard conditions
standard_conditions = {
    'flood_depth': 1.5,
    'building_type': 'residential',
    'area_m2': 100
}

print("🌍 Global Regional Comparison")
print("=" * 80)
print(f"Standard conditions: {standard_conditions['flood_depth']}m depth, {standard_conditions['building_type']}, {standard_conditions['area_m2']}m²")
print()
print(f"{'City':12} | {'Country':12} | {'Region':20} | {'Max €/m²':10} | {'Damage':12} | {'Ratio':6}")
print("-" * 80)

regional_results = []

for location in global_comparison:
    result = calculator.calculate_jrc_damage(
        latitude=location['lat'],
        longitude=location['lon'],
        country_code=location['code'],
        **standard_conditions
    )
    
    region = result['location']['region']
    max_damage_per_m2 = result['property_characteristics']['max_damage_per_m2_eur']
    damage = result['damage_assessment']['economic_damage_eur']
    ratio = result['damage_assessment']['damage_ratio']
    
    regional_results.append({
        'city': location['name'],
        'country': location['country'],
        'region': region,
        'max_damage_per_m2': max_damage_per_m2,
        'damage': damage,
        'ratio': ratio
    })
    
    print(f"{location['name']:12} | {location['country']:12} | {region:20} | €{max_damage_per_m2:8.0f} | €{damage:9,.0f} | {ratio:5.1%}")

# Calculate statistics
damages = [r['damage'] for r in regional_results]
print("-" * 80)
print(f"{'STATISTICS':12} | {'':12} | {'':20} | {'':10} | {'':12} | {'':6}")
print(f"{'Average':12} | {'':12} | {'':20} | {'':10} | €{np.mean(damages):9,.0f} | {'':6}")
print(f"{'Range':12} | {'':12} | {'':20} | {'':10} | €{min(damages):4,.0f}-{max(damages):4,.0f} | {'':6}")
print(f"{'Std Dev':12} | {'':12} | {'':20} | {'':10} | €{np.std(damages):9,.0f} | {'':6}")

In [ ]:
# Visualize regional comparison
df_regional = pd.DataFrame(regional_results)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Global Regional Comparison - Residential Buildings (1.5m flood, 100m²)', fontsize=14, fontweight='bold')

# 1. Damage by city
bars1 = ax1.bar(df_regional['city'], df_regional['damage'], color='steelblue', alpha=0.7)
ax1.set_title('Economic Damage by City')
ax1.set_ylabel('Damage (EUR)')
ax1.tick_params(axis='x', rotation=45)

# Add value labels
for bar, value in zip(bars1, df_regional['damage']):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 500,
             f'€{value:,.0f}', ha='center', va='bottom', fontsize=8)

# 2. Damage ratio by region
region_summary = df_regional.groupby('region').agg({
    'damage': 'mean',
    'ratio': 'mean'
}).reset_index()

bars2 = ax2.bar(region_summary['region'], region_summary['ratio'], color='lightcoral', alpha=0.7)
ax2.set_title('Average Damage Ratio by Region')
ax2.set_ylabel('Damage Ratio')
ax2.tick_params(axis='x', rotation=45)
ax2.set_ylim(0, 1)

# 3. Max damage per m² comparison
ax3.scatter(df_regional['max_damage_per_m2'], df_regional['damage'], s=100, alpha=0.7, color='green')
for i, city in enumerate(df_regional['city']):
    ax3.annotate(city, (df_regional['max_damage_per_m2'].iloc[i], df_regional['damage'].iloc[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)
ax3.set_xlabel('Max Damage per m² (EUR)')
ax3.set_ylabel('Actual Damage (EUR)')
ax3.set_title('Max Damage vs Actual Damage')

# 4. Regional distribution pie chart
region_counts = df_regional['region'].value_counts()
ax4.pie(region_counts.values, labels=region_counts.index, autopct='%1.0f%%', startangle=90)
ax4.set_title('Cities by Region')

plt.tight_layout()
plt.show()

## 9. Practical Use Cases {#use-cases}

Let's explore some real-world applications of the flood damage library.

In [ ]:
# Use Case 1: Flood Risk Assessment for Insurance
print("🏦 Use Case 1: Insurance Risk Assessment")
print("=" * 60)
print("Scenario: Insurance company evaluating flood risk for a commercial property in London")
print()

# Property details
london_property = {
    'latitude': 51.5074,
    'longitude': -0.1278,
    'country_code': 'GB',
    'building_type': 'commercial',
    'area_m2': 500
}

# Different flood scenarios with probabilities
flood_scenarios = [
    {'name': 'Minor flood (10-year)', 'depth': 0.5, 'annual_probability': 0.10},
    {'name': 'Moderate flood (25-year)', 'depth': 1.0, 'annual_probability': 0.04},
    {'name': 'Major flood (100-year)', 'depth': 2.0, 'annual_probability': 0.01},
    {'name': 'Extreme flood (500-year)', 'depth': 3.0, 'annual_probability': 0.002}
]

print(f"{'Scenario':25} | {'Depth':5} | {'Prob.':6} | {'Damage':12} | {'Expected Loss':12}")
print("-" * 65)

total_expected_annual_loss = 0

for scenario in flood_scenarios:
    result = calculator.calculate_jrc_damage(
        latitude=london_property['latitude'],
        longitude=london_property['longitude'],
        flood_depth=scenario['depth'],
        country_code=london_property['country_code'],
        building_type=london_property['building_type'],
        area_m2=london_property['area_m2']
    )
    
    damage = result['damage_assessment']['economic_damage_eur']
    expected_loss = damage * scenario['annual_probability']
    total_expected_annual_loss += expected_loss
    
    print(f"{scenario['name']:25} | {scenario['depth']:4.1f}m | {scenario['annual_probability']:5.1%} | €{damage:9,.0f} | €{expected_loss:9,.0f}")

print("-" * 65)
print(f"{'TOTAL EXPECTED ANNUAL LOSS':25} | {'':5} | {'':6} | {'':12} | €{total_expected_annual_loss:9,.0f}")

# Calculate suggested premium
risk_margin = 1.3  # 30% margin for uncertainty and profit
suggested_premium = total_expected_annual_loss * risk_margin

print(f"\n💡 Insurance Recommendations:")
print(f"  Expected Annual Loss: €{total_expected_annual_loss:,.0f}")
print(f"  Suggested Annual Premium: €{suggested_premium:,.0f} (with 30% margin)")
print(f"  Premium as % of property value: {suggested_premium/result['property_characteristics']['total_value_eur']:.2%}")

In [ ]:
# Use Case 2: Investment Location Comparison
print("🏗️ Use Case 2: Investment Location Comparison")
print("=" * 60)
print("Scenario: Real estate developer choosing between locations for a shopping center")
print()

# Potential locations
investment_locations = [
    {'name': 'Amsterdam', 'country': 'Netherlands', 'lat': 52.37, 'lon': 4.90, 'code': 'NL'},
    {'name': 'Hamburg', 'country': 'Germany', 'lat': 53.55, 'lon': 9.99, 'code': 'DE'},
    {'name': 'Venice', 'country': 'Italy', 'lat': 45.44, 'lon': 12.32, 'code': 'IT'},
    {'name': 'Rotterdam', 'country': 'Netherlands', 'lat': 51.92, 'lon': 4.48, 'code': 'NL'}
]

# Shopping center specifications
shopping_center = {
    'building_type': 'commercial',
    'area_m2': 2000,
    'flood_depth': 1.5  # Design flood scenario
}

print(f"Shopping center: {shopping_center['area_m2']}m² commercial building")
print(f"Design flood scenario: {shopping_center['flood_depth']}m depth")
print()
print(f"{'Location':12} | {'Country':12} | {'Region':16} | {'Property Value':14} | {'Flood Damage':12} | {'Risk Level':10}")
print("-" * 85)

location_analysis = []

for location in investment_locations:
    result = calculator.calculate_jrc_damage(
        latitude=location['lat'],
        longitude=location['lon'],
        country_code=location['code'],
        **shopping_center
    )
    
    region = result['location']['region']
    property_value = result['property_characteristics']['total_value_eur']
    damage = result['damage_assessment']['economic_damage_eur']
    damage_ratio = result['damage_assessment']['damage_ratio']
    
    # Risk classification
    if damage_ratio < 0.3:
        risk_level = "Low"
    elif damage_ratio < 0.6:
        risk_level = "Medium"
    else:
        risk_level = "High"
    
    location_analysis.append({
        'name': location['name'],
        'country': location['country'],
        'region': region,
        'property_value': property_value,
        'damage': damage,
        'damage_ratio': damage_ratio,
        'risk_level': risk_level
    })
    
    print(f"{location['name']:12} | {location['country']:12} | {region:16} | €{property_value:11,.0f} | €{damage:9,.0f} | {risk_level:10}")

# Recommendation
best_location = min(location_analysis, key=lambda x: x['damage_ratio'])
print(f"\n🏆 Recommendation: {best_location['name']} has the lowest flood risk ({best_location['damage_ratio']:.1%} damage ratio)")

In [ ]:
# Use Case 3: Cost-Benefit Analysis of Flood Protection
print("🛡️ Use Case 3: Flood Protection Cost-Benefit Analysis")
print("=" * 70)
print("Scenario: Industrial facility in Rotterdam evaluating flood protection measures")
print()

# Facility details
rotterdam_facility = {
    'latitude': 51.9225,
    'longitude': 4.4792,
    'country_code': 'NL',
    'building_type': 'industrial',
    'area_m2': 3000
}

# Protection scenarios
protection_scenarios = [
    {'name': 'No protection', 'flood_depth': 2.5, 'protection_cost': 0},
    {'name': 'Basic barriers', 'flood_depth': 2.0, 'protection_cost': 100000},
    {'name': 'Advanced barriers', 'flood_depth': 1.5, 'protection_cost': 300000},
    {'name': 'Comprehensive system', 'flood_depth': 1.0, 'protection_cost': 600000}
]

print(f"Industrial facility: {rotterdam_facility['area_m2']}m² in Rotterdam, Netherlands")
print()
print(f"{'Protection Level':20} | {'Cost':10} | {'Flood Depth':11} | {'Damage':12} | {'Net Benefit':11} | {'ROI':6}")
print("-" * 75)

# Calculate baseline (no protection)
baseline_result = calculator.calculate_jrc_damage(
    latitude=rotterdam_facility['latitude'],
    longitude=rotterdam_facility['longitude'],
    flood_depth=protection_scenarios[0]['flood_depth'],
    country_code=rotterdam_facility['country_code'],
    building_type=rotterdam_facility['building_type'],
    area_m2=rotterdam_facility['area_m2']
)
baseline_damage = baseline_result['damage_assessment']['economic_damage_eur']

protection_analysis = []

for scenario in protection_scenarios:
    result = calculator.calculate_jrc_damage(
        latitude=rotterdam_facility['latitude'],
        longitude=rotterdam_facility['longitude'],
        flood_depth=scenario['flood_depth'],
        country_code=rotterdam_facility['country_code'],
        building_type=rotterdam_facility['building_type'],
        area_m2=rotterdam_facility['area_m2']
    )
    
    damage = result['damage_assessment']['economic_damage_eur']
    damage_avoided = baseline_damage - damage
    net_benefit = damage_avoided - scenario['protection_cost']
    roi = (net_benefit / scenario['protection_cost'] * 100) if scenario['protection_cost'] > 0 else 0
    
    protection_analysis.append({
        'name': scenario['name'],
        'cost': scenario['protection_cost'],
        'depth': scenario['flood_depth'],
        'damage': damage,
        'damage_avoided': damage_avoided,
        'net_benefit': net_benefit,
        'roi': roi
    })
    
    print(f"{scenario['name']:20} | €{scenario['protection_cost']:8,.0f} | {scenario['flood_depth']:8.1f}m | €{damage:9,.0f} | €{net_benefit:8,.0f} | {roi:5.1f}%")

# Find optimal solution
optimal = max(protection_analysis, key=lambda x: x['net_benefit'])
print(f"\n🎯 Optimal solution: {optimal['name']} with net benefit of €{optimal['net_benefit']:,.0f}")

## 10. Best Practices {#best-practices}

Here are some best practices and tips for using the flood damage library effectively.

In [ ]:
print("📋 Best Practices for Flood Damage Calculations")
print("=" * 60)
print()

print("1. 🎯 Data Quality and Validation")
print("   ✅ Always validate input coordinates")
print("   ✅ Use appropriate country codes when available")
print("   ✅ Verify building types match your use case")
print("   ✅ Consider local economic conditions")
print()

print("2. 📊 Uncertainty and Risk Assessment")
print("   ✅ Always include uncertainty analysis")
print("   ✅ Use confidence intervals for decision making")
print("   ✅ Consider multiple flood scenarios")
print("   ✅ Validate results with local data when possible")
print()

print("3. 🌍 Regional Considerations")
print("   ✅ Understand regional differences in damage functions")
print("   ✅ Consider economic development levels")
print("   ✅ Account for local building standards")
print("   ✅ Be aware of data limitations in some regions")
print()

print("4. 💰 Economic Adjustments")
print("   ✅ Adjust for inflation (data is from 2010)")
print("   ✅ Consider local currency fluctuations")
print("   ✅ Account for local construction costs")
print("   ✅ Include indirect economic impacts when relevant")
print()

print("5. 🔄 Batch Processing Tips")
print("   ✅ Use batch processing for large datasets")
print("   ✅ Handle errors gracefully in batch operations")
print("   ✅ Monitor memory usage for very large datasets")
print("   ✅ Consider parallel processing for performance")
print()

In [ ]:
# Example of proper error handling and validation
def safe_damage_calculation(calculator, location_data):
    """
    Example of safe damage calculation with proper error handling.
    """
    try:
        # Validate inputs
        lat = location_data.get('latitude')
        lon = location_data.get('longitude')
        depth = location_data.get('flood_depth')
        
        if not all([lat is not None, lon is not None, depth is not None]):
            return {'error': 'Missing required parameters'}
        
        if not (-90 <= lat <= 90):
            return {'error': f'Invalid latitude: {lat}'}
        
        if not (-180 <= lon <= 180):
            return {'error': f'Invalid longitude: {lon}'}
        
        if depth < 0:
            return {'error': f'Invalid flood depth: {depth}'}
        
        # Perform calculation
        result = calculator.calculate_jrc_damage(
            latitude=lat,
            longitude=lon,
            flood_depth=depth,
            country_code=location_data.get('country_code'),
            building_type=location_data.get('building_type', 'residential'),
            area_m2=location_data.get('area_m2', 100)
        )
        
        return result
        
    except Exception as e:
        return {'error': f'Calculation failed: {str(e)}'}

# Test the safe calculation function
print("🛡️ Example of Safe Calculation with Error Handling")
print("=" * 55)

test_cases = [
    {'name': 'Valid case', 'latitude': 52.52, 'longitude': 13.40, 'flood_depth': 1.5, 'country_code': 'DE'},
    {'name': 'Invalid latitude', 'latitude': 200, 'longitude': 13.40, 'flood_depth': 1.5},
    {'name': 'Negative depth', 'latitude': 52.52, 'longitude': 13.40, 'flood_depth': -1.0},
    {'name': 'Missing depth', 'latitude': 52.52, 'longitude': 13.40}
]

for test_case in test_cases:
    result = safe_damage_calculation(calculator, test_case)
    
    if 'error' in result:
        print(f"❌ {test_case['name']:15}: {result['error']}")
    else:
        damage = result['damage_assessment']['economic_damage_eur']
        print(f"✅ {test_case['name']:15}: €{damage:,.2f}")

In [ ]:
# Performance comparison: individual vs batch processing
import time

print("⚡ Performance Comparison: Individual vs Batch Processing")
print("=" * 60)

# Generate test data
np.random.seed(42)
n_locations = 50

test_locations = []
for i in range(n_locations):
    test_locations.append({
        'latitude': np.random.uniform(40, 60),  # Europe roughly
        'longitude': np.random.uniform(-10, 30),
        'flood_depth': np.random.uniform(0.5, 3.0),
        'building_type': np.random.choice(['residential', 'commercial', 'industrial']),
        'area_m2': np.random.randint(50, 500)
    })

# Method 1: Individual calculations
start_time = time.time()
individual_results = []
for location in test_locations:
    try:
        result = calculator.calculate_jrc_damage(**location)
        individual_results.append(result)
    except:
        individual_results.append({'error': 'Calculation failed'})
individual_time = time.time() - start_time

# Method 2: Batch processing
start_time = time.time()
batch_results = calculator.calculate_damage_batch_jrc(test_locations)
batch_time = time.time() - start_time

print(f"📊 Results for {n_locations} locations:")
print(f"  Individual processing: {individual_time:.3f} seconds")
print(f"  Batch processing:      {batch_time:.3f} seconds")
print(f"  Speed improvement:     {individual_time/batch_time:.1f}x faster")
print(f"  Time per location:     {batch_time/n_locations*1000:.1f} ms")

# Verify results are consistent
successful_individual = len([r for r in individual_results if 'error' not in r])
successful_batch = len([r for r in batch_results if 'error' not in r])

print(f"\n✅ Successful calculations:")
print(f"  Individual: {successful_individual}/{n_locations}")
print(f"  Batch:      {successful_batch}/{n_locations}")

print(f"\n💡 Recommendation: Use batch processing for {n_locations//10}+ locations")

## Summary

This tutorial has covered the comprehensive use of the Flood Damage Calculation Library with JRC data. You've learned:

### Key Takeaways:

1. **Basic Usage**: How to calculate flood damage for individual locations using JRC data
2. **Data Understanding**: The structure and coverage of JRC damage functions and maximum damage values
3. **Advanced Features**: Building type comparisons, automatic region inference, and batch processing
4. **Uncertainty Analysis**: How to interpret and use confidence intervals in damage estimates
5. **Regional Comparisons**: Understanding how damage functions vary across different global regions
6. **Practical Applications**: Real-world use cases including insurance, investment analysis, and protection planning
7. **Best Practices**: Error handling, performance optimization, and data validation

### Next Steps:

- **Explore your own data**: Apply the library to your specific locations and scenarios
- **Validate with local data**: Compare results with local damage assessments when available
- **Consider economic adjustments**: Update values for inflation and local economic conditions
- **Integrate with other tools**: Combine with GIS systems, hydrological models, or risk assessment frameworks

### Resources:

- **Documentation**: See README.md for complete API reference
- **Source Code**: Explore the library source code for customization
- **JRC Database**: Original data from Joint Research Centre (2017)
- **Tests**: Run the test suite to verify functionality

The library provides a solid foundation for flood damage assessment, but remember that results should be interpreted within the context of local conditions and validated against available data when possible.